In [1]:
%reload_ext autoreload
%autoreload 2

## 内容 

### 1. [Basic Node](http://116.85.5.40:8118/notebooks/docs/schema/k12ai_basic_type.ipynb)

### 2. [Complex Node](http://116.85.5.40:8118/notebooks/docs/schema/k12ai_complext_type.ipynb)

### 3. [Layout Node](http://116.85.5.40:8118/notebooks/docs/schema/k12ai_layout_type.ipynb)

In [2]:
from k12libs.utils.nb_easy import k12ai_print
from k12libs.utils.nb_easy import k12ai_get_app_dir
from k12libs.utils.nb_easy import k12ai_get_top_dir
from k12libs.utils.nb_widget import k12ai_schema_parse

from IPython.display import Markdown, JSON, display

import os
import json
import _jsonnet

In [3]:
show_default = True
show_schema_node = show_default
show_basic_jsonnet = show_default
show_basic_json = show_default
parse_cv_schema = False

# K12 Data Template Model Define

In [4]:
schema_dir = os.path.join(k12ai_get_top_dir(), 'k12libs', 'templates', 'schema')
!ls $schema_dir 

basic.jsonnet


## 1 k12框架配置模板, 采用jsonnet方式

> Jsonnet配置文件仅k12框架内部配置使用, 需要转换为json提供给前端.

In [5]:
file_md = os.path.join(k12ai_get_top_dir(), 'docs', 'schema_node.md')
if show_basic_jsonnet:
    display(Markdown(filename = file_md))


## Schema Note Tree

```
                                                   page
                                                    |
                                                    |
                                    +---------------+----------------+
                                    |                                |
                      object        v                                v
                         ^         tab                              tab
                         |          |
                         |          |
             +----------------------+--------------------------+----------------+
             |                      |                          |                |
             |                      |                          |                |
             v                      v                          v                v
          accordion            navigation      object     <<k12type>>      <<layout>>
             |                      |            ^                              |
             |                      |            |                              |
             |                      |            |                  +-----------+-----------+
     +-------+------+      +--------+--------+--------------+       |           |           |
     |              |      |                 |              |       |           |           |
     |              |      |                 |              |       v           v           v
     |              |      v                 v              v       H           HV          V
     |              |  <<k12type>>      <<k12type>>    <<layout>>
     v              v                        |
<<k12type>>    navigation                    |
                                             |
                            +----------------+------------------+
                            |                    |              |
                            v                    |              v
      link             <<basictype>>             v       <<complextype>>
        ^                   |                  object           |
        |                   |                                   |
        |                   |                                   |
    +--------+-------+------+-------+---------+                 +---> bool-trigger
    |        |       |              |         |                 |
    |        |       |              |         |                 |
    v        v       v              v         v                 +---> string-enum-trigger
int[-array] bool  float[-array]  string   string-enum           |
                                                                |
                                                                +---> string-enum-array-trigger
                                                                |
                                                                |
                                                                +---> string-enum-group-trigger

```

**符号"<< >>"的节点是虚节点(不存在), 只是为了描述树图**

1. 一个节点树中只能还有一个page节点, 且page节点的子节点只能是tab节点, 可以是多个tab.
2. 一个tab节点可以包含多个除page以外的其他任意节点.
3. 一个navigation节点可以包含多个`"<<k12type>>"`节点. (实际上可以嵌套包含accordion/navigation, 但不推荐)
4. `<<layout>>`节点可以用来指定该节点下的内容布局方式, 如: 横向(H), 竖向(V), 基础类型和复杂类型分层(HV)
5. `<<basictype>>` 基本类型(叶子节点), 包括 int, float, string等, 节点特点简单只是用来存值
6. `<<complextype>>` 复杂类型, 包括 bool-trigger(当值为true时, 触发新的配置项/节点),
   string-enum-trigger(当选中其中一个值时会触发相应的配置项/节点), ...


## Node Struct

### 一般节点模型

```json
{
    "description": "",  // [O] 字符串, 节点描述
    "_id_": "a.b.c",    // [C] 字符串, 中间节点是可选[O], 叶子几点必选[M], 其值有重要意义, 用来还原配置
    "name": {},         // [O] 对象, 形如: {en:"test", cn:"测试"}
    "type": "page",     // [M] 字符串, 节点的类型
    "objs": [],         // [M] 对象数组, 子节点内容
}
```

### 叶子节点模型

A. 布尔型(bool)

```json
{
    "_id_": "a.b.c",   // [C] 字符串, 中间节点是可选[O], 叶子几点必选[M], 其值有重要意义, 用来还原配置
    "name": {},        // [O] 对象, 形如: {en:"test", cn:"测试"}
    "type": "bool",    // [M] 字符串, 节点的类型
    "default": false,  // [M] 布尔型, 默认值
}
```

B. 整型(int)

```json
{
    "_id_": "a.b.c",   // [C] 字符串, 中间节点是可选[O], 叶子几点必选[M], 其值有重要意义, 用来还原配置
    "name": {},        // [O] 对象, 形如: {en:"test", cn:"测试"}
    "type": "int",     // [M] 字符串, 节点的类型
    "min": 1,          // [O] 整型, 最大可设置的值
    "max": 100,        // [O] 整型, 最小可设置的值
    "default": 10,     // [M] 整型, 默认值
}
```

C. 浮点型(float)

```json
{
    "_id_": "a.b.c",   // [C] 字符串, 中间节点是可选[O], 叶子几点必选[M], 其值有重要意义, 用来还原配置
    "name": {},        // [O] 对象, 形如: {en:"test", cn:"测试"}
    "type": "float",   // [M] 字符串, 节点的类型
    "min": 1.0,        // [O] 浮点型, 最大可设置的值
    "max": 100.0,      // [O] 浮点型, 最小可设置的值
    "default": 10.0,   // [M] 浮点型, 默认值
}
```

D. 文本型(string), 文本枚举(string-enum)

```json
{
    "_id_": "a.b.c",   // [C] 字符串, 中间节点是可选[O], 叶子几点必选[M], 其值有重要意义, 用来还原配置
    "name": {},        // [O] 对象, 形如: {en:"test", cn:"测试"}
    "type": "string",  // [M] 字符串, 节点的类型
    "default": "text", // [M] 字符串, 默认值
}
```


In [6]:
# basic schema template file
basic_file = os.path.join(schema_dir, 'basic.jsonnet')
if show_basic_jsonnet:
    !cat $basic_file

// @file basic.jsonnet
// @brief
// @author QRS
// @version 1.0
// @date 2019-12-18 21:08

local default_schema = {
    bool(id): {
        _id_: 'bool-' + id,
        name: { en: 'Bool' + id, cn: '布尔-' + id },
        type: 'bool',
        default: true,
    },

    int(id): {
        _id_: 'int-' + id,
        name: { en: 'Int-' + id, cn: '整型-' + id },
        type: 'int',
        min: 0,
        max: 120,
        default: 100,
    },

    float(id): {
        _id_: 'float-' + id,
        name: { en: 'Float' + id, cn: '浮点-' + id },
        type: 'float',
        min: 0,
        max: 300,
        default: 200.0,
    },

    string(id): {
        _id_: 'string-' + id,
        name: { en: 'String-' + id, cn: '字符串-' + id },
        type: 'value',
        default: 'string',
    },

    string_enum(id): {
        _id_: 'stringenum-' + id,
        name: { en: 'StringEnum-' + id, cn: '字符串枚举-' + id },
        type: 'string-enum',
        objs: [
            {
                name: { en: 'Item

转换为json:

In [7]:
basic_json = _jsonnet.evaluate_file(basic_file)
if show_basic_json:
    k12ai_print(basic_json, indent=2)

{
  "description": "K12 Data Template Model\n",
  "objs": [
    {
      "name": {
        "cn": "页标签-1",
        "en": "tab1"
      },
      "objs": [
        {
          "name": {
            "cn": "折叠卡-1",
            "en": "accordion1"
          },
          "objs": [
            {
              "_id_": "tab1.accordion1.navigation1",
              "default": "nav1-1",
              "name": {
                "cn": "导航栏-1",
                "en": "navigation1"
              },
              "objs": [
                {
                  "name": {
                    "cn": "选项卡-1",
                    "en": "group1"
                  },
                  "objs": [
                    {
                      "_id_": "bool-1",
                      "default": true,
                      "name": {
                        "cn": "布尔-1",
                        "en": "Bool1"
                      },
                      "type": "bool"
                    },
                    {
             

解析json生成配置界面:

In [8]:
k12ai_schema_parse(json.loads(basic_json), lan='cn', debug=False);

Box(children=(VBox(children=(Tab(children=(VBox(children=(Accordion(children=(VBox(children=(VBox(children=(To…

### 实例说明

#### 1.  页标签-1中包含3个折叠卡节点, 折叠卡节点的特点是最多只能有一个能够伸展开, jsonnet配置如下

```json
{
    name: { en: 'tab1', cn: '页标签-1' },
    type: 'tab',
    objs: [
        {
            name: { en: 'accordion1', cn: '折叠卡-1' },
            type: 'accordion',
            objs: [],
        },
        {
            name: { en: 'accordion2', cn: '折叠卡-2' },
            type: 'accordion',
            objs: [],
        },
        {
            name: { en: 'accordion3', cn: '折叠卡-3' },
            type: 'accordion',
            objs: [],
        },
    ],
}
```

#### 2. 折叠卡-1中包含2个导航卡节点, 导航卡节点和折叠卡是相似的, 展示形式有区别, 导航卡jsonnet配置如下:

```json
{
    name: { en: 'navigation1', cn: '导航栏-1' },
    type: 'navigation',
    objs: [
        {
            name: { en: 'group1', cn: '选项卡-1' }, // 导航选项卡显示的名字 
            type: 'object',       // 如果类型为"object",且name存在, 可以描述子节点的标题(黑体+粗体) 
            objs: [],
        },
        {
            name: { en: 'group2', cn: '选项卡-2' }, // 导航选项卡显示的名字 
            type: '_ignore_',    // 忽略该节点标题(可以是任意非定义的关键字符), 只需解析它的子节点即可
            objs: [],
        },
    ],  // objs
}
```

> `object`和`_ignore_`可以在任意节点中使用

#### 3. 折叠卡-2和页标签-3中包含基本数据类型的节点

>基本数据类型(int, float...)以及复杂数据类型(xxx-trigger)可以存在与任意一个节点下(page节点除外)

#### 4. 页标签-2中包含1个导航卡节点

> navigation父节点可以是accordion节点, 也可以是tab节点

#### 5. 页标签-3中除布尔型节点, 其他节点水平排列(type='H), 且能自动折行, jsonnet配置如下

```json
{
    name: { en: 'tab3', cn: '页标签-3' },
    type: 'tab',
    objs: [
        default_schema.bool('5'),           // 使用上一级的布局放置该元素
        {
            type: 'H',                      // "H"节点的子节点采用自动折行水平展示, 如上图"页标签-3"
            objs: [
                default_schema.int('5'),
                default_schema.float('5'),
                default_schema.string('5'),
                default_schema.int('6'),
                default_schema.float('6'),
                default_schema.string('6'),
            ],
        },
    ],
},
```

#### 6. 页标签-3中同时包含基本类型的数组节点

> 当成普通的字符串节点, 控制逻辑需自行实现数组和字符串之间的转换, 如果不是数组形式的字符串, 转换过程会异常.

## CV 实例

In [9]:
cv_schema_dir = os.path.join(k12ai_get_app_dir('cv'), 'templates', 'schema')
cv_json = _jsonnet.evaluate_file(os.path.join(cv_schema_dir, 'k12cv.jsonnet'))
# k12ai_print(cv_json)

In [10]:
if parse_cv_schema:
    k12ai_schema_parse(json.loads(cv_json), debug=False)
    